In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/chendasiv/software2023.git

fatal: destination path 'software2023' already exists and is not an empty directory.


-------------------最初にここまで実行-------------------

_dicに人名の画像ファイル、_imgに判定したい画像ファイルをアップロードしてください

ファイルの追加、削除はいつでも行えます

ファイルのアップロードが終了したら次のセルを実行してください

In [ ]:
# face.py
import face_recognition
import numpy as np
from PIL import Image, ImageDraw

import glob
import os

dic_dir = 'software2023/_dic/'
unk_dir = 'software2023/_img/'

def face_func(dic_dir, unk_dir):
    #print('Start')
    #print(dic_dir)
    #print(unk_dir)
    dic = sorted(glob.glob(dic_dir + '/*.jpg'))
    unk = sorted(glob.glob(unk_dir + '/*.jpg')) #unknown
    known_face_encodings = []
    known_face_names = []

    for img in dic:
        #print('Loop1')
        name = os.path.splitext(os.path.basename(img))[0]
        people_image = face_recognition.load_image_file(img)
        if len(face_recognition.face_encodings(people_image)) != 0:
          #print('len(face_recognition.face_encodings(people_image))')
          people_encoding = face_recognition.face_encodings(people_image)[0]

          known_face_encodings.append(people_encoding)
          known_face_names.append(name)

        else :
          print('Error')
          print(img + ':Error')

    for img in unk:
        #print('Loop2')
        #print(img)
        unknown_image = face_recognition.load_image_file(img)

        face_locations = face_recognition.face_locations(unknown_image)
        face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

        pil_image = Image.fromarray(unknown_image)

        draw = ImageDraw.Draw(pil_image)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            #print('Loop3')
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                #print('matches[best_match_index]')
                name = known_face_names[best_match_index]

            #print(name)

            pil_image.show()

face_func(dic_dir, unk_dir)

RuntimeError: ignored